<a href="https://colab.research.google.com/github/paragmilmile24/Algoflex/blob/master/LU%20decomposition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%writefile input.c
//Initial code to make binary files
#include<stdio.h>
#include<stdlib.h>
void main()
{
    FILE *input;
    if((input = fopen("/content/in.dat","ab+")) == NULL)
    {
        printf("Error! opening input file");
        exit(1);
    }
    float A[3][3]= {0};
    float B[3]= {0};
    int i,j,n;
    printf("Enter the order of square matrix: ");
    scanf("%d",&n);
    printf("\nEnter matrix element:\n");
    for(i=0; i<n; i++)
    {
        for(j=0; j<n; j++)
        {
            printf("Enter A[%d][%d] element: ", i,j);
            scanf("%f",&A[i][j]);
            fwrite(&A[i][j],sizeof(A[i][j]),1,input);
        }
    }
    printf("\nEnter the constant terms: \n");
    for(i=0; i<n; i++)
    {
        printf("B[%d]: ",i);
        scanf("%f",&B[i]);
        fwrite(&B[i],sizeof(B[i]),1,input);
    }
    fclose(input);
}

Writing input.c


In [2]:
%%shell
gcc input.c -o infile
./infile

Enter the order of square matrix: 3

Enter matrix element:
Enter A[0][0] element: 1
Enter A[0][1] element: 1
Enter A[0][2] element: 1
Enter A[1][0] element: 4
Enter A[1][1] element: 3
Enter A[1][2] element: -1
Enter A[2][0] element: 3
Enter A[2][1] element: 5
Enter A[2][2] element: 3

Enter the constant terms: 
B[0]: 1
B[1]: 6
B[2]: 4


In [3]:
%%writefile LU_decomposition_method.c
//LU Decomposition Algorithm
#include<stdio.h>
#include<stdlib.h>
//AX=B  Consider A=LU where L is lower Triangular matrix
//U is upper triangular matrix
// LUX=B Let Y=UX hence LY=B

void main()
{
    FILE *input,*output;
    if((input = fopen("/content/in.dat","ab+")) == NULL)
    {
        printf("Error! opening input file");
        exit(1);
    }
    if((output = fopen("/content/out.dat","ab+")) == NULL)
    {
        printf("Error! opening output file");
        exit(1);
    }
    float A[3][3]= {0},L[3][3]= {0}, U[3][3];
    float B[3]= {0}, X[3]= {0},Y[3]= {0};
    int i,j,k,n;
    printf("Enter the order of square matrix: ");
    scanf("%d",&n);
    //reading input from file
    fseek(input, 0, SEEK_SET);
    for(i=0; i<n; i++)
    {
        for(j=0; j<n; j++)
        {
            fread(&A[i][j], sizeof(float), 1, input);
        }
    }
    for(i=0; i<n; i++)
    {
        fread(&B[i], sizeof(float), 1, input);
    }

   //calculating upper triangular matrix
   for (int i = 0; i < n; i++)
	{
		// Upper Triangular
		for (int k = i; k < n; k++)
		{
			// Summation of L(i, j) * U(j, k)
			int sum = 0;
			for (int j = 0; j < i; j++)
				sum += (L[i][j] * U[j][k]);

			// Evaluating U(i, k)
			U[i][k] = A[i][k] - sum;
		}
  
		// Lower Triangular
		for (int k = i; k < n; k++)
		{
			if (i == k)
	       L[i][i] = 1; // Diagonal as 1
			else
			{
				// Summation of L(k, j) * U(j, i)
				int sum = 0;
				for (int j = 0; j < i; j++)
					sum += (L[k][j] * U[j][i]);

				// Evaluating L(k, i)
				L[k][i]
					= (A[k][i] - sum) / U[i][i];
			}
		}
	}
    printf("Lower triangular matrix\n[L]: \n");
    for(i=0; i<n; i++)
    {
        for(j=0; j<n; j++)
            printf("%9.3f",L[i][j]);
        printf("\n");
    }
    printf("\n\nUpper triangular matrix\n[U]: \n");
    for(i=0; i<n; i++)
    {
        for(j=0; j<n; j++)
            printf("%9.3f",U[i][j]);
        printf("\n");
    }
    for(i=0; i<n; i++)
    {
        Y[i]=B[i];
        for(j=0; j<i; j++)
        {
            Y[i]-=L[i][j]*Y[j];
        }
    }
    printf("\n\n[Y]: \n");
    for(i=0; i<n; i++)
    {
        printf("%9.3f",Y[i]);
    }
    for(i=n-1; i>=0; i--)
    {
        X[i]= Y[i];
        for(j=i+1; j<n; j++)
        {
            X[i]-=U[i][j]*X[j];
        }
        X[i]/=U[i][i];
    }
    printf("\n\nOUTPUT\n[X]: \n");
    for(i=0; i<n; i++)
    {
        printf("%9.3f",X[i]);
        fwrite(&X[i],sizeof(X[i]),1,output);
    }
    fclose(input);
    fclose(output);
}


Writing LU_decomposition_method.c


In [4]:
%%shell
gcc LU_decomposition_method.c -o answer
./answer

Enter the order of square matrix: 3
Lower triangular matrix
[L]: 
    1.000    0.000    0.000
    4.000    1.000    0.000
    3.000   -2.000    1.000


Upper triangular matrix
[U]: 
    1.000    1.000    1.000
    0.000   -1.000   -5.000
   -0.000    0.000  -10.000


[Y]: 
    1.000    2.000    5.000

OUTPUT
[X]: 
    1.000    0.500   -0.500